In [22]:
import numpy as np
import imageio
import cv2
import glob
import os

import open3d as o3d
from pyquaternion import Quaternion
import matplotlib.pyplot as plt

from MatlabTests.python.read_write_model import *
from MatlabTests.python.read_dense import *

## Colmap sparse and dense results

In [19]:
input_model = "TestMap/sparse"
input_format = ".bin"
image_folder = "TestMap/images/"
depth_folder = "TestMap/mvs/tv_depths/"

In [7]:
cameras, images, points3D = read_model(path=input_model, ext=input_format)
print("num_cameras:", len(cameras))
print("num_images:", len(images))
print("num_points3D:", len(points3D))

num_cameras: 351
num_images: 351
num_points3D: 27470


![mesh colmap](images/mesh_colmap.png)

In [9]:
def q_t_to_matrix(q_vec, t_vec):
    quaternion = Quaternion(q_vec)
    pose = np.eye(4)
    pose[0:3,0:3] = quaternion.rotation_matrix
    pose[0:3, 3] = t_vec
    #pose = np.linalg.inv(pose)
    return pose

## TSDF Mapping

In [72]:
def reshape_depth(depth, size=(480,640)):
    reshaped_depth = np.zeros(size)
    height = min(size[0], depth.shape[0])
    width = min(size[1], depth.shape[1])
    reshaped_depth[0:height, 0:width] = depth[0:height, 0:width]
    return reshaped_depth

def filter_depth(depth, prec = 5):
    # filter by the percentile
    depth_tmp = np.zeros(depth.shape) + depth
    depth_range = np.percentile(depth_tmp[depth_tmp>0.1], [prec,100-prec])

    depth_tmp[depth_tmp < depth_range[0]] = 0
    depth_tmp[depth_tmp > depth_range[1]] = 0
    
    # filter the closest points
    max_t = np.max(depth_tmp[depth_tmp> 0.01])
    min_t = np.min(depth_tmp[depth_tmp> 0.01])
    
    threshold = (max_t - min_t)/50 + min_t
    depth[depth < threshold] = 0
    return depth

In [89]:
def show_rgbd_test(rgbd_image):
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image,
        o3d.camera.PinholeCameraIntrinsic(
            o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))
    # Flip it, otherwise the pointcloud will be upside down
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    
    plt.subplot(1, 2, 1)
    plt.title('color image')
    plt.imshow(rgbd_image.color)
    plt.subplot(1, 2, 2)
    plt.title('depth image')
    plt.imshow(rgbd_image.depth)
    plt.show()
    return pcd

In [88]:
volume = o3d.integration.ScalableTSDFVolume(
    voxel_length=2.0 / 512.0,
    sdf_trunc=0.02,
    color_type=o3d.integration.TSDFVolumeColorType.RGB8)

count = 1
for image_id, Image in images.items():
    pose_camera = q_t_to_matrix(Image.qvec, Image.tvec*0.1) #np.linalg.inv
    image_file = image_folder + "/" + Image.name
    depth_file = depth_folder + "/" + Image.name.split("/")[-1] + ".geometric.bin"
    if(not os.path.exists(depth_file)):
        continue
    
    color = o3d.io.read_image(image_file)
    depth_py = filter_depth(read_array(depth_file))
    depth_py = reshape_depth(depth_py)*100
    
    depth = o3d.geometry.Image(depth_py.astype('u2'))
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
                color, depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
    
    camera_param = cameras[Image.camera_id].params
    volume.integrate(
        rgbd_image,
        o3d.camera.PinholeCameraIntrinsic(640, 480, camera_param[0], camera_param[0], camera_param[1], camera_param[2]),
        pose_camera)
    print("\r current process frame :["+str(count)+"/"+str(len(images))+"]", end="", flush= True)
    count += 1

 current process frame :[351/351]

In [90]:
print("Extract a triangle mesh from the volume and visualize it.")
mesh = volume.extract_triangle_mesh()
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

Extract a triangle mesh from the volume and visualize it.


![image mesh](images/mesh_test.png)

In [86]:
o3d.io.write_triangle_mesh("test_mesh.ply", mesh)

True